In [5]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()




In [4]:
!pip install tensorflow

     -------------------------------------- 272.8/272.8 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     -------------------------------------- 440.7/440.7 kB 9.2 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 6.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.to

In [6]:
data = pd.read_csv("news.csv")
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
data.tail(20)

,Unnamed: 0,title,text,label
6315,3717,Police Arrest Suspect In Charleston Church Sho...,"Police in Charleston, S.C., say a man they sus...",REAL
6316,5205,Donald Trump’s collapse was caused by one big ...,Silver of FiveThirtyEight.com has laid out fou...,REAL
6317,6696,FINA suspends Russian swimmer for 8 years over...,This post was originally published on this sit...,FAKE
6318,7991,BREAKING : Hillary Campaign Manager Deletes hi...,BREAKING : Hillary Campaign Manager Deletes hi...,FAKE
6319,1303,Why Ted Cruz Has the Most to Lose in New Hamps...,Ted Cruz took first prize in the Iowa caucuses...,REAL
6320,9051,“Nothing Good Can Come of This Election”–and T...,"Posted on November 4, 2016 by Charles Hugh Smi...",FAKE
6321,10200,List of Republicans opposing Trump | OffGuardian,"Charlie Baker , Massachusetts (2015–present)[3...",FAKE
6322,10009,Putin: Use of 'mythical' Russian military thre...,"vladimir putin , Valdai , sochi , RBTH Daily R...",FAKE
6323,4214,Bernie Sanders says private meeting with Pope ...,ROME — U.S. Democratic presidential candidate...,REAL
6324,2316,Alabama Lawmaker: Same-Sex Couples Don’t Deser...,Most conservatives who oppose marriage equalit...,REAL


In [9]:
#describe the number of roes and coloumns  in the data set 
data.shape

(6335, 4)

In [10]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [11]:
#Data encoding can be done
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

#it converts the categorical values into numerical values


In [12]:

embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [13]:
#Now tokenization can be don
import nltk
from nltk.tokenize import word_tokenize

In [14]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

In [15]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
	sequences1, padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]


In [19]:
embeddings_index = {}
with open('glove.6B.50d.txt',"r",encoding='utf-8') as f:
	for line in f:
		values = line.split()
		word = values[0]
		coefs = np.asarray(values[1:], dtype='float32')
		embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embeddings_matrix[i] = embedding_vector


In [20]:
model = tf.keras.Sequential([
	tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
							input_length=max_length, weights=[
								embeddings_matrix],
							trainable=False),
	tf.keras.layers.Dropout(0.2),
	tf.keras.layers.Conv1D(64, 5, activation='relu'),
	tf.keras.layers.MaxPooling1D(pool_size=4),
	tf.keras.layers.LSTM(64),
	tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
			optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 50)            377600    
                                                                 
 dropout (Dropout)           (None, 54, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            16064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                        

In [21]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
					epochs=num_epochs,
					validation_data=(testing_padded,
									testing_labels),
					verbose=2)


Train on 2700 samples, validate on 300 samples
Epoch 1/50


C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 3s - loss: 0.6509 - accuracy: 0.6104 - val_loss: 0.5761 - val_accuracy: 0.6767 - 3s/epoch - 1ms/sample
Epoch 2/50
2700/2700 - 2s - loss: 0.5748 - accuracy: 0.7019 - val_loss: 0.5935 - val_accuracy: 0.6567 - 2s/epoch - 559us/sample
Epoch 3/50
2700/2700 - 2s - loss: 0.5354 - accuracy: 0.7333 - val_loss: 0.5181 - val_accuracy: 0.7333 - 2s/epoch - 657us/sample
Epoch 4/50
2700/2700 - 2s - loss: 0.4894 - accuracy: 0.7652 - val_loss: 0.4988 - val_accuracy: 0.7300 - 2s/epoch - 614us/sample
Epoch 5/50
2700/2700 - 2s - loss: 0.4521 - accuracy: 0.7893 - val_loss: 0.4754 - val_accuracy: 0.7667 - 2s/epoch - 611us/sample
Epoch 6/50
2700/2700 - 1s - loss: 0.4038 - accuracy: 0.8196 - val_loss: 0.4695 - val_accuracy: 0.7567 - 1s/epoch - 543us/sample
Epoch 7/50
2700/2700 - 2s - loss: 0.3649 - accuracy: 0.8400 - val_loss: 0.5185 - val_accuracy: 0.7200 - 2s/epoch - 600us/sample
Epoch 8/50
2700/2700 - 2s - loss: 0.3344 - accuracy: 0.8563 - val_loss: 0.5047 - val_accuracy: 0.7533 - 2s/epoch - 62

In [23]:
# sample text to check if fake or not
X = "Watch The Exact Moment Paul Ryan Committed Pol"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
						padding=padding_type,
						truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
	print("This news is True")
else:
	print("This news is false")


This news is false
